## Dependencies

In [417]:
import sys
import os
sys.path.append('C:\\Users\oscar\OneDrive\Dokumenter\Høst 2023\TET4565 Spesialiseringsemne\Hydro_optimization') #OSCAR path
sys.path.append('C:\\Users\\benny\\Documents\\Preliminary-project')  #BENJAMIN path
sys.path.append('C:\\Users\\Epsilon Delta\\OneDrive - NTNU\\Energi og Miljø NTNU\\Høst2023\\TET4565 Fordypningsemne\\Hydro_optimization') #ESPEN path

import pyomo.environ as pyo
import numpy as np
from pyomo.environ import ConcreteModel,Set,RangeSet,Param,Suffix,Reals,NonNegativeReals,NonPositiveReals,Binary,Objective,minimize,maximize,value
from pyomo.core import Constraint,Var,Block,ConstraintList
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
from pyomo.util.infeasible import log_infeasible_constraints
from pyomo.environ import ConstraintList
from pyomo.dae import *
import matplotlib.pyplot as plt
from calculations.datahandling import*
from calculations.data_processor import* 
from scipy.interpolate import interp1d
import itertools


## Reading in parameters and topology data

In [418]:
# ---------------------------- Read in parameters and hydro topology -------------------------------------- # WE WAIT WITH THIS UNTIL LATER

def InputParameter(data_file):
    inputdata = pd.read_excel(data_file)
    inputdata = inputdata.set_index('Parameter', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['prod'] = inputdata[['Ci', 'yi', 'P_min', 'P_max']]
    data['prod'].drop('Solar', inplace=True)
    return data

parameters=InputParameter('data/Parameters.xlsx')


def InputTopology(data_file):
    inputdata = pd.read_excel(data_file, sheet_name='Hydro')
    inputdata = inputdata.set_index('Ormsetfoss', drop=True)
    inputdata = inputdata.transpose()
    data = {}
    data['ormset'] = inputdata[['Pmin', 'Pmax','Vmax','Vmin','Vstart','Qmin','Qmax','Pump_min','Pump_max', 'Q','LRL','HRL','RL_start', 'Spill_min', 'Spill_max']].dropna()
    return data 
topology = InputTopology('data/Ormset_Data.xlsx')

    #Volume-head relationship:
def InputHead(data_file):
    inputdata = pd.read_excel(data_file, sheet_name='Vol_head')
    volume_column='Mm3'
    head_column='m'
    data_volhead = dict(zip(inputdata[volume_column], inputdata[head_column]))
    return data_volhead
headvol_Ormset=InputHead('data/Ormset_Data.xlsx')

def InputHead2(data_file):
    inputdata = pd.read_excel(data_file, sheet_name='Vol_head2')
    volume_column='Mm3'
    head_column='m'
    data_volhead = dict(zip(inputdata[volume_column].dropna(), inputdata[head_column].dropna()))
    return data_volhead
headvol_Buaset=InputHead2('data/Ormset_Data.xlsx')

    #Flow-head relationship:
def InputFlow(data_file): 
    inputdata = pd.read_excel(data_file, sheet_name='flow')
    data_flowhead = {}
    data_flowhead['ormset_headflow'] = inputdata[['Moh','m3/s']].dropna()
    return data_flowhead
headflow=InputFlow('data/Ormset_Data.xlsx')

display(topology)
display(headvol_Ormset)
display(headflow)

def interpolate_head(volume, head_dict):
    volumes = list(head_dict.keys())
    heads = list(head_dict.values())
    return np.interp(volume, volumes, heads)

test=interpolate_head(45, headvol_Ormset)
print(test)


{'ormset': Ormsetfoss  Pmin  Pmax  Vmax  Vmin  Vstart  Qmin  Qmax  Pump_min  Pump_max  \
 Hydro1       0.0  43.0  45.0   0.0    45.0   0.0  12.0       0.0       0.0   
 Hydro2       0.0  30.0  20.0   0.0    20.0   0.0  12.0       0.0       1.6   
 
 Ormsetfoss    Q    LRL    HRL  RL_start  Spill_min  Spill_max  
 Hydro1      4.0  375.0  389.0     389.0        0.0       12.0  
 Hydro2      2.0  326.0  331.5     331.5        0.0       12.0  }

{0.0: 375.0,
 2.08333: 375.99242,
 4.16667: 376.83333,
 6.25: 377.63462,
 8.33333: 378.40476,
 10.41667: 379.09314,
 12.5: 379.70588,
 14.58333: 380.30093,
 16.66667: 380.87963,
 18.75: 381.45833,
 20.83333: 382.03509,
 22.91667: 382.58333,
 25.0: 383.13158,
 27.08333: 383.67982,
 29.16667: 384.22807,
 31.25: 384.77632,
 33.33333: 385.32456,
 35.41667: 385.85417,
 37.5: 386.35714,
 39.58333: 386.85317,
 41.66667: 387.33333,
 43.75: 387.79348,
 45.83333: 388.24638,
 47.91667: 388.65278,
 50.0: 389.0}

{'ormset_headflow':      Moh  m3/s
 0  388.0   0.0
 1  388.2   4.9
 2  388.5  20.4
 3  389.0  68.4}

388.0652204347847


In [419]:
start = '2018-04-01 00:00:00'
end = '2018-04-01 23:00:00'
start_dt = pd.to_datetime(start)
end_dt = pd.to_datetime(end)

    #Input data and read-in
input_data_market = read_csv_data('data/Market_price.csv')              #MARKET
market_prices_h=convert_to_dict(input_data_market, start, end, 'H') ##avg_market_price=average_value(market_prices_h)
avg_market_price=average_value(market_prices_h)

input_data_inflow = read_csv_data('data/Data_inflow.csv')
columns = ['Ormsetvatn','Buavatn']
inflow = convert_to_dict(input_data_inflow, start, end, 'H', columns)
inflow_hydro1 = inflow[0]  # First dictionary for Hydro1 - ORMSETVATN
inflow_hydro2 = inflow[1]  # Second dictionary for Hydro2 - BUAVATN

hours_difference = ((end_dt - start_dt).total_seconds() / 3600)+1  # 3600 seconds in an hour

## Functions


In [420]:


# Define your input data and constants
#L = {1: 30, 2: 20, 3: 20, 4: 30, 5: 50, 6: 80, 7: 50, 8: 90, 9: 110, 10: 150, 11: 120, 12: 80, 13: 70, 14: 80, 15: 90, 16: 160, 17: 170, 18: 150, 19: 120, 20: 100, 21: 70, 22: 60, 23: 50, 24: 40}

constants = {'eff': 0.91, 'rho': 1000, 'g': 9.81, 'Ri': 100, 'Si': 300, 
             'inflow_scenarios' : ['High', 'Avg', 'Low'], 
             'inflow_probabilities' : {'High': 1/3, 'Avg': 1/3, 'Low': 1/3}, 
             'states': {100: 1, 75: 0.75, 50: 0.5, 25: 0.25, 0:0 }}




# Variables for power produced or bought and volume, water level etc.
def p_bounds(model, i, j, s, v1, v2):
    return (model.Pmin[i], model.Pmax[i])

def q_bounds(model, i, j, s, v1, v2): #flow rate
    return (model.Qmin[i], model.Qmax[i])

def pump_bounds(model, i, j, s, v1, v2):
    return(model.Pump_min[i], model.Pump_max[i])

def vol_bounds(model, i, j, s, v1, v2):
    return (model.Vmin[i], model.Vmax[i])

def init_v(model, i, j, s, v1, v2):
    return model.v0[i]

def head_bounds(model,i,j, s ,v1, v2):
    return (model.LRL[i], model.HRL[i])

def spill_bounds(model, i, j, s, v1, v2):
    return (model.S_min[i], model.S_max[i])

#initializing scenarios for inflow

def inflow1_scenario(model, i, j, s):
    if i=='Hydro1':
        if s == 'High': 
            return model.inflow[i,j,s]==scale_dict(inflow_hydro1, 1.5)[j]
        elif s == 'Avg':
            return model.inflow[i,j,s]==scale_dict(inflow_hydro1, 1)[j]
        elif s == 'Low':
            return model.inflow[i,j,s]==scale_dict(inflow_hydro1, 0.5)[j]
    elif i == 'Hydro2':
        if s == 'High': 
            return model.inflow[i,j,s]==scale_dict(inflow_hydro2, 1.5)[j]
        elif s == 'Avg':
            return model.inflow[i,j,s]==scale_dict(inflow_hydro2, 1)[j]
        elif s == 'Low':
            return model.inflow[i,j,s]==scale_dict(inflow_hydro2, 0.5)[j]


#def cumulative_pump_rule(model, i, j, s, v):
    #if j == 1:
        #if i == 'Hydro1':
           #return model.cumulative_pump[i, j, s, v] == 0
        #elif i == 'Hydro2':
            #return model.cumulative_pump[i,j, s, v] == model.pump[i,j, s, v]
    #else:
       #if i =='Hydro1':
        #    return model.cumulative_pump[i, j, s, v]  == 0
        #elif i =='Hydro2':
        #    return model.cumulative_pump[i, j, s, v]  == model.cumulative_pump[i, j - 1, s, v] + model.pump[i,j, s, v]


#def cumulative_q_rule(model, i, j, s, v):
 #   if j == 1:
  #      if i == 'Hydro1':
   #         return model.cumulative_q[i, j, s, v] == model.q[i, j, s, v] 
    #    elif i == 'Hydro2':
     #     return model.cumulative_q[i,j, s, v] == model.q[i,j, s, v] 
    #else:
     #   if i =='Hydro1':
      #      return model.cumulative_q[i, j, s, v]  == model.cumulative_q[i, j - 1, s, v] + model.q[i, j, s, v]
       # elif i =='Hydro2':
        #    return model.cumulative_q[i, j, s, v]  == model.cumulative_q[i, j - 1, s, v] + model.q[i, j, s, v] 


#def cumulative_inflow_rule(model, i, j, s):
 #   if j == 1:
  #      if i == 'Hydro1':
   ##         return model.cumulative_inflow[i, j, s] == model.inflow[i, j, s] 
     #   else:  # Hydro2
      #      return model.cumulative_inflow[i, j, s] == model.inflow[i, j, s] 
    #else:
     #   if i == 'Hydro1':
      #      return model.cumulative_inflow[i, j, s] == model.cumulative_inflow[i, j - 1, s] + model.inflow[i, j, s]
       # else:  # Hydro2
        #    return model.cumulative_inflow[i, j, s] == model.cumulative_inflow[i, j - 1, s] + model.inflow[i, j, s]

# Constraints
def load_rule(model, j, s, v1, v2):
    return model.p['Hydro1',j, s, v1, v2] + model.p['Hydro2',j, s, v1, v2] + model.m['DAM',j, s, v1, v2] == model.L[j] - (model.r[j, s, v1, v2])

def power_rule(model,i, j, s, v1, v2):   #Rule for hydro power production dependent on varying discharge #update to also vary on head
    return  10**-6*model.eff * model.rho * model.g * model.HRL[i] * model.q[i, j, s, v1, v2] == model.p[i, j, s, v1, v2]#MW

def vol_rule(model, i, s, v1, v2):
    tolerance = 1e1  # Adjust the tolerance value as needed (so that volume does not HAVE to be full if we simulate over a small time period)
    return model.v[i, min(model.periods), s, v1, v2] >= model.v0[i] - tolerance

# Hydro parameters
fm3 = 3600/1000000 # Conversion factor between m^3/s and Mm^3/hour. 1h = 3600s and M = 10^6.
# Constraint to update volume based on cumulative sum of discharges (VOLUME [MM3] - DISCHARGE [MM3/h] + INFLOW [MM3/h])
# Constraint to update volume based on cumulative sum of discharges (VOLUME [MM3] - DISCHARGE [MM3/h] + INFLOW [MM3/h])


#def v_init_rule(model, i, v):
   #return model.v_init[i,v]==model.v0[i]*v/100
   

   
def alpha_function_rule(model, j, s, v1, v2):
    return model.exp[j, s, v1, v2] == (model.probs[s] * (model.Ri[j] * ( model.r[j, s, v1, v2]) + model.Mi[j] * (model.m['DAM', j, s, v1, v2])))
    
def generate_alpha(model, j, s, v1, v2):
    step=10
    if j == max(model.periods):
        if v1 and v2==100:
            a = model.exp[j, s, v1, v2]
            return model.alpha[j-1, s, v1, v2] >= a
        elif  v2==100:
            a = model.exp[j, s, v1, v2]
            b_numerator= ((model.exp[j, s, v1+step,v2]-model.exp[j,s,v1,v2]))
            b_denominator=((model.v['Hydro1', j, s, v1+step, v2])+(model.v['Hydro2', j, s,  v1+step, v2])-((model.v['Hydro1', j, s, v1, v2])+(model.v['Hydro2', j, s,  v1, v2])))
            return model.alpha[j-1, s, v1, v2] >= a + b_numerator/b_denominator*((model.v['Hydro1', j, s, v1, v2]+model.v['Hydro1', j, s, v1, v2]))
        elif v1==100:
            a = model.exp[j, s, v1, v2]
            b_numerator= ((model.exp[j, s, v1,v2+step]-model.exp[j,s,v1,v2]))
            b_denominator=((model.v['Hydro1', j, s, v1, v2+step])+(model.v['Hydro2', j, s,  v1, v2+step])-((model.v['Hydro1', j, s, v1, v2])+(model.v['Hydro2', j, s,  v1, v2])))
            return model.alpha[j-1, s, v1, v2] >= a + b_numerator/b_denominator*((model.v['Hydro1', j, s, v1, v2]+model.v['Hydro1', j, s, v1, v2]))

        else:
            epsilon=1e-6
            a = model.exp[j,s,v1, v2]
            b_numerator= ((model.exp[j, s, v1+step,v2+step]-model.exp[j,s,v1,v2]))
            b_denominator=((model.v['Hydro1', j, s, v1+step, v2+step])+(model.v['Hydro2', j, s,  v1+step, v2+step])-((model.v['Hydro1', j, s, v1, v2])+(model.v['Hydro2', j, s,  v1, v2])))
            
            # Add the linearized alpha constraint using 'w'
            return model.alpha[j-1, s, v1, v2] >= a  + b_numerator/b_denominator*((model.v['Hydro1', j, s, v1, v2]+model.v['Hydro1', j, s, v1, v2]))

    elif min(model.periods) + 1 < j < max(model.periods):
        if v1 and v2==100:
            a = model.exp[j, s, v1, v2]
            return model.alpha[j-1, s, v1, v2] >= a
        elif  v2==100:
            a = model.exp[j, s, v1, v2]
            b_numerator= ((model.exp[j, s, v1+step,v2]-model.exp[j,s,v1,v2]))
            b_denominator=((model.v['Hydro1', j, s, v1+step, v2])+(model.v['Hydro2', j, s,  v1+step, v2])-((model.v['Hydro1', j, s, v1, v2])+(model.v['Hydro2', j, s,  v1, v2])))
            return model.alpha[j-1, s, v1, v2] >= a + b_numerator/b_denominator*((model.v['Hydro1', j, s, v1, v2]+model.v['Hydro1', j, s, v1, v2]) - (model.v['Hydro1', j+1, s, v1, v2])+(model.v['Hydro2', j+1, s,  v1, v2]))
        elif v1==100:
            a = model.exp[j, s, v1, v2]
            b_numerator= ((model.exp[j, s, v1,v2+step]-model.exp[j,s,v1,v2]))
            b_denominator=((model.v['Hydro1', j, s, v1, v2+step])+(model.v['Hydro2', j, s,  v1, v2+step])-((model.v['Hydro1', j, s, v1, v2])+(model.v['Hydro2', j, s,  v1, v2])))
            return model.alpha[j-1, s, v1, v2] >= a + b_numerator/b_denominator*((model.v['Hydro1', j, s, v1, v2]+model.v['Hydro1', j, s, v1, v2]) - (model.v['Hydro1', j+1, s, v1, v2])+(model.v['Hydro2', j+1, s,  v1, v2]))

        else:
            epsilon=1e-6
            a = model.exp[j,s,v1, v2]
            b_numerator= ((model.exp[j, s, v1+step,v2+step]-model.exp[j,s,v1,v2]))
            b_denominator=((model.v['Hydro1', j, s, v1+step, v2+step])+(model.v['Hydro2', j, s,  v1+step, v2+step])-((model.v['Hydro1', j, s, v1, v2])+(model.v['Hydro2', j, s,  v1, v2])))
            
            # Add the linearized alpha constraint using 'w'
            return model.alpha[j-1, s, v1, v2] >= a  + b_numerator/b_denominator*((model.v['Hydro1', j, s, v1, v2]+model.v['Hydro1', j, s, v1, v2]) - (model.v['Hydro1', j+1, s, v1, v2])+(model.v['Hydro2', j+1, s,  v1, v2]))
                
    else:
        return model.alpha[j,s,v1,v2]== model.exp[j,s, 100, 100]
    
   
def initial_volume_rule1(model, i, v1, v2):
    if i == 'Hydro1':
        return model.v0[i]*v1/100
    elif i == 'Hydro2':
        return model.v0[i]*v2/100


    
def volume_change_rule(model, i, j, s, v1, v2):
    if j>1:   
        if i == 'Hydro1':
            return model.v[i, j , s, v1, v2] == (model.v_init[i,v1, v2]) - (model.q[i, j, s, v1, v2] *fm3) + (model.inflow[i, j, s]*fm3) +(model.pump['Hydro2',j, s, v1, v2]*fm3) -(model.s[i,j,s,v1, v2]*fm3)
        elif i == 'Hydro2':
        # Include the discharge from Hydro1 as inflow to Hydro2 
            return model.v[i, j, s, v1, v2] == (model.v_init[i,v1, v2]) - (model.q[i, j, s, v1, v2] *fm3) + (model.inflow[i, j, s]*fm3) + (model.q['Hydro1', j, s, v1, v2]*fm3) -(model.s[i,j,s,v1, v2]*fm3)
    else: 
        if i == 'Hydro1':
            return model.v[i, j , s, v1, v2] == (model.v0[i]) - (model.q[i, j, s, v1, v2] *fm3) +(model.pump['Hydro2',j, s, v1, v2]*fm3) -(model.s[i,j,s,v1, v2]*fm3)
        elif i == 'Hydro2':
    #    # Include the discharge from Hydro1 as inflow to Hydro2 
            return model.v[i, j, s, v1, v2] == (model.v0[i]) - (model.q[i, j, s, v1, v2] *fm3) + (model.q['Hydro1', j, s, v1, v2]*fm3) -(model.s[i,j,s,v1, v2]*fm3)
         
    
def OBJ_one_stage(model):
    total_sum = 0.0
    for j in model.periods:
        for v1, v2 in model.states:
            for s in model.scenarios:
            
                if j>1:
                    total_sum += model.probs[s] * (model.Ri[j] * model.r[j, s, v1, v2] + model.Mi[j] * model.m['DAM', j, s, v1, v2] + model.alpha[j-1, s, v1, v2])
                else:
                    total_sum += model.probs[s] * (model.Ri[j] * model.r[j, s, v1, v2] + model.Mi[j] * model.m['DAM', j, s, v1, v2] + model.alpha[j, s, v1, v2])

    return total_sum




def create_matrix(model):
    matrix=[]
    for state in model.states:
        row=[]
        for stage in model.periods:
            v=model.alpha['Hydro1', stage, 'High', state].value
            row.append(v)
        matrix.append(row)
    return matrix


## Solve one stage setup


In [421]:
def solve_one_stage(constants, topology, stages, states):
    # Create a ConcreteModel
    model = pyo.ConcreteModel(name="SDP")

    # Define the inflow scenarios and associated probabilities
    model.scenarios=pyo.Set(initialize=constants['inflow_scenarios'])
    
    #Defining reservoir states:
    #jump=5
    #states1= [i for i in range(model.Vmin['Hydro1'], model.Vmax['Hydro1'] + jump, jump)]
    #states2= [i for i in range(model.Vmin['Hydro2'], model.Vmax['Hydro2'] + jump, jump)]
    
    Hydro1_initial=states
    Hydro2_initial=states
    
    state_combinations =[p for p in itertools.product(Hydro1_initial, Hydro2_initial)]
    
    model.states=pyo.Set(initialize=state_combinations)
    
    # Sets to model hydro topology
    model.periods = pyo.Set(initialize=stages)
    #model.periods = pyo.Set(initialize=range(1, 25, 1))                   #these sets must be defined in the latex report as well
    model.plants = pyo.Set(initialize=['Hydro1', 'Hydro2'])               #uses i
    model.market = pyo.Set(initialize=['DAM','Sell'])


    # Hydro parameters
    model.eff = pyo.Param(initialize=constants['eff'])
    model.rho = pyo.Param(initialize=constants['rho'])
    model.g = pyo.Param(initialize=constants['g'])

    
    # Cost parameters
    #max_purchase_limit = 100  # Adjust this value as needed                #Buy-in market price (initial market price cost)
    #model.PurchaseLimit = pyo.Param(initialize=max_purchase_limit)
    model.Mi = pyo.Param(model.periods, initialize=avg_market_price)            #Market price varying
    model.Ri=pyo.Param(model.periods, initialize=constants['Ri'])  #Rationing cost for plants
    model.Si=pyo.Param(model.plants, initialize=constants['Si'])  #Spillage  costs for plants 

    #Plant parameters
    model.Pmin=pyo.Param(model.plants, initialize=topology['ormset']['Pmin'])
    model.Pmax=pyo.Param(model.plants, initialize=topology['ormset']['Pmax'])

    #Load parameters
    model.L = pyo.Param(model.periods, initialize={j: market_prices_h[j]*5 for j in model.periods})

    #Topology parameters
    model.Qmin=pyo.Param(model.plants, initialize=topology['ormset']['Qmin']) 
    model.Qmax=pyo.Param(model.plants, initialize=topology['ormset']['Qmax'])
    model.Pump_min=pyo.Param(model.plants, initialize=topology['ormset']['Pump_min'])
    model.Pump_max=pyo.Param(model.plants, initialize=topology['ormset']['Pump_max'])
    # model.Vmin=pyo.Param(model.plants, initialize=topology['ormset']['Vmin'])
    model.Vmin=pyo.Param(model.plants, initialize=topology['ormset']['Vmin'])
    model.Vmax=pyo.Param(model.plants, initialize=topology['ormset']['Vmax'])
    model.v0 = pyo.Param(model.plants, initialize=topology['ormset']['Vstart'])
    model.LRL =pyo.Param(model.plants, initialize=topology['ormset']['LRL'])
    model.HRL =pyo.Param(model.plants, initialize=topology['ormset']['HRL'])
    model.S_min=pyo.Param(model.plants, initialize=topology['ormset']['Spill_min'])
    model.S_max=pyo.Param(model.plants, initialize=topology['ormset']['Spill_max'])
    
    
    model.v_init=pyo.Param(model.plants, model.states, initialize=initial_volume_rule1)
    #model.v_init2=pyo.Param(model.plants, model.states, initialize=initial_volume_rule2)

    #Scenario parameter
    model.probs=pyo.Param(model.scenarios, initialize=constants['inflow_probabilities'])
    
    
    # --- Variables ----
    model.inflow = pyo.Var(model.plants, model.periods, model.scenarios,  within=NonNegativeReals)
    model.p = pyo.Var(model.plants, model.periods, model.scenarios, model.states,   bounds=p_bounds)                   #Power production
    model.m = pyo.Var(model.market, model.periods, model.scenarios, model.states, within=NonNegativeReals)       #Power bougth from lack of production
    model.q = pyo.Var(model.plants, model.periods, model.scenarios,  model.states, bounds=q_bounds)
    model.q_out=pyo.Var(model.plants, model.periods, model.scenarios,  model.states, within=NonNegativeReals)
    model.pump=pyo.Var(model.plants, model.periods, model.scenarios, model.states, bounds=pump_bounds)     #Time dependent pump discharge between lower and upper reservoir (dependent on reservoir balance)
    model.v = pyo.Var(model.plants, model.periods, model.scenarios, model.states, within=NonNegativeReals, initialize= init_v) #time dependent volume which varies based on discharge over time (convert from m3/s to MM3/day ellerno og subtraher)
    #model.h = pyo.Var(model.plants, model.periods, model.scenarios,  model.states, bounds=head_bounds)
    model.r=pyo.Var(model.periods, model.scenarios, model.states, within=NonNegativeReals) #amount rationed 
    model.s=pyo.Var(model.plants, model.periods, model.scenarios, model.states, bounds=spill_bounds) #amount spilled for each reservoir (should have bounds)
    
    #future cost variable 
    model.exp=pyo.Var(model.periods, model.scenarios, model.states, within=NonNegativeReals)
    model.alpha=pyo.Var(model.periods, model.scenarios, model.states, within=NonNegativeReals)
    
    # additional linearization variable:
    #model.y=pyo.Var()
    model.w = Var(within=NonNegativeReals)
    
        #-------------------MODEL OF RESERVOIRS------------------#
    model.cumulative_q = pyo.Var(model.plants, model.periods, model.scenarios, model.states,  within=pyo.NonNegativeReals)
    model.cumulative_inflow = pyo.Var(model.plants, model.periods, model.scenarios, within=pyo.NonNegativeReals)#combining discharge (q [m3/s]) into m3 over given timeframe (NEED TO CHECK CONVERSIONS)
    model.cumulative_pump = pyo.Var(model.plants, model.periods, model.scenarios, model.states, within=pyo.NonNegativeReals)
    # Constraint to calculate the cumulative sum of discharges (WATER OUT OF PLANTS)
 
    #--- CONSTRAINTS---

    model.inflow_cons=pyo.Constraint(model.plants, model.periods, model.scenarios,  rule=inflow1_scenario)
   #model.cumulative_pump_cons = pyo.Constraint(model.plants, model.periods, model.scenarios, model.states, rule=cumulative_pump_rule)
    #model.cumulative_q_cons = pyo.Constraint(model.plants, model.periods, model.scenarios, model.states,  rule=cumulative_q_rule)
    #model.cumulative_inflow_cons = pyo.Constraint(model.plants, model.periods, model.scenarios, rule=cumulative_inflow_rule)
    model.load_cons = pyo.Constraint(model.periods, model.scenarios, model.states,  rule=load_rule)
    model.power_prod = pyo.Constraint(model.plants, model.periods, model.scenarios, model.states, rule=power_rule)
    #model.volume_end = pyo.Constraint(model.plants, model.scenarios, model.states,  rule=vol_rule)    

    #model.vol_state_cons=pyo.Constraint(model.plants, model.states, rule=v_init_rule)
    
    model.alpha_func=pyo.Constraint(model.periods, model.scenarios, model.states, rule=alpha_function_rule)
                
    model.generate_alpha=pyo.Constraint(model.periods, model.scenarios, model.states, rule=generate_alpha)
    
    model.volume_change_cons = pyo.Constraint(model.plants, model.periods, model.scenarios, model.states)
    # --- Objective function---
    model.obj=pyo.Objective(rule=OBJ_one_stage(model), sense=pyo.minimize)
    
    
        
    return model
    


def solve(model):
    #solver 
    opt = SolverFactory('gurobi', solver_io="python") 
    #opt.options['NonConvex']=2    #defining dual 
    model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
    results = opt.solve(model,tee=True) 
            
    return results, model

def displayresults(model):
    return print(model.display()) #model.dual.display())

def print_volume(model):
    for j in model.periods:
        for v1, v2 in model.states:
            print(model.v['Hydro1', j, 'High', v1,v2], ':', model.v['Hydro1', j, 'High', v1, v2].value)

In [422]:
states=list(range(100,-1, -10))
stages=reversed(range(1, int(hours_difference) + 1, 1))



m=solve_one_stage(constants, topology, stages, states)
solve(m)
displayresults(m)

print_volume(m)




DegreeError: GurobiDirect does not support expressions of degree None.
expr: exp[3,Avg,0,0] + (exp[3,Avg,10,10] - exp[3,Avg,0,0])/(v[Hydro1,3,Avg,10,10] + v[Hydro2,3,Avg,10,10] - (v[Hydro1,3,Avg,0,0] + v[Hydro2,3,Avg,0,0]))*(v[Hydro1,3,Avg,0,0] + v[Hydro1,3,Avg,0,0] - v[Hydro1,4,Avg,0,0] + v[Hydro2,4,Avg,0,0]) - alpha[2,Avg,0,0]

In [ ]:
#return model.alpha[j, s, v1, v2] >= (((model.v['Hydro1',j, s, v1-25, v2-25]+model.v['Hydro2', j,s, v1-25, v2-25])-(model.v['Hydro1',j, s, v1, v2]+model.v['Hydro2', j,s, v1, v2]))*model.exp[j, s,v1, v2] + ((model.v['Hydro1',j, s, v1, v2]+model.v['Hydro2', j,s, v1, v2])-(model.v['Hydro1',j, s, v1+25, v2+25]+model.v['Hydro2', j,s, v1+25, v2+25]))*model.exp[j, s,v1-25, v2-25])/((model.v['Hydro1',j, s, v1-25, v2-25]+model.v['Hydro2', j,s, v1-25, v2-25])-(model.v['Hydro1',j, s, v1+25, v2+25]+model.v['Hydro2', j,s, v1+25, v2+25]))